In [1]:
include("setup.jl")

Total number of nodes on fault = 121
Average node spacing = 0.0004
dt = 0.00007477186690791
Setup Complete


In [ ]:
# Plot friction parameters
using Gadfly

fricPlot = plot(layer(x = (cca-ccb).*1e3 + 10, y = FltX,Theme(default_color=colorant"red"), Geom.point),
                layer(x = Seff/1e6, y = FltX, Theme(default_color=colorant"deepskyblue"), Geom.point),
                layer(x = tauo/1e6, y = FltX, Theme(default_color=colorant"green"), Geom.point),
                Guide.xlabel("Scaled (a-b)/ Stress value"),
                Guide.ylabel("Depth (m)"),
                Guide.title("Rate and state friction/Stress"),
                Coord.Cartesian(ymin=-24))

In [ ]:
include("timesolver.jl")

In [ ]:
iglob[:,:,1]

In [ ]:
const aqw = 2
const swq = 2

In [ ]:
const sss = swq*aqw

In [3]:
FaultNglob

121